## Simple source modeling
Trying to iron out all the confusion on how to calculate source parameters

In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

t, a, b, c, f = sym.symbols('t a b c f')

## Source Model

We will use a simple guassian as a source model.

In [ ]:
# Get gausian and derivative/integral
gaus = a * sym.exp((-(t - b)**2) / (2 * c **2))
d_gaus = sym.diff(gaus, t)
int_gaus = sym.integrate(gaus, t)

In [ ]:
# get freq domain
F_gaus = sym.fourier_transform(gaus, t, f).rewrite(sym.Integral)
F_d_gaus = sym.fourier_transform(d_gaus, t, f).rewrite(sym.Integral)
F_int_gaus = sym.fourier_transform(int_gaus, t, f).rewrite(sym.Integral)

In [ ]:
gaus

In [ ]:
d_gaus

## Get numpy functions

In [ ]:
get_gaus = sym.lambdify([t, a, b, c], gaus)
get_d_gaus = sym.lambdify([t, a, b, c], d_gaus)
get_int_gaus = sym.lambdify([t, a, b, c], int_gaus)

## Next generate source time series and plot

In [ ]:
# get source in 1) displacement 2) velocity
dt = 0.01
t1, t2 = 2, 8
a, b, c = 0.1, 5, np.sqrt(2)
x = np.arange(0, 10, dt)


source_disp = get_gaus(x, a=a, b=b, c=c)
# source_disp = np.zeros(len(source_disp)+20)
source_disp[len(source_disp)//2:len(source_disp)] = 0
source_vel = np.gradient(source_disp, dt) #get_d_gaus(x, a=a, b=b, c=c)

In [ ]:
plt.plot(x, source_disp)
plt.xlabel('time (s)')
plt.ylabel('displacement amplitude (m)')

In [ ]:
plt.plot(x, source_vel)
plt.xlabel('time (s)')
plt.ylabel('velocity amplitude (m)')

In [ ]:
len(source_disp)

## Moments
First we estimate the moment (or something proporcional to it) in time domain and compare the results to the freq domain.

### Time domain

In [ ]:
# get moment from t1 to t2
x_in_time = (x > t1) & (x < t2)

In [ ]:
moment_td = np.trapz(source_disp, dx=x[1] - x[0])

### Freq. domain

In [ ]:
def get_fft(ar, dt):
    """Return the fft of the array and its """
    fft_ar = np.fft.rfft(ar) * dt
    freq = np.fft.rfftfreq(len(ar), dt)
    return freq, fft_ar

In [ ]:
freqs, source_disp_fft = get_fft(source_disp, dt)

In [ ]:
plt.loglog(freqs, abs(source_disp_fft))
plt.xlabel('frequency (hz)')
plt.ylabel('amplitude')

In [ ]:
moment_fd = abs(source_disp_fft)[0]

In [ ]:
print(moment_td, moment_fd)

## Energy
Next we estimate energy and the equivalent in the freq domain.

## Time domain energy

In [ ]:
source_energy = source_vel ** 2

In [ ]:
plt.plot(x, source_energy)
plt.xlabel('time (s)')
plt.ylabel('amplitude (m/s)')

In [ ]:
# energy_td = np.trapz(source_energy, dx=dt)
energy_td = np.sum(source_energy)

## Frequency domain energy

In [ ]:
def get_ppsd(ar, dt):
    """Get the power spectral density? Or something close to it ;)"""
    print(ar.min(), ar.max())
    fft = np.fft.rfft(ar)
    print(abs(fft).min(), abs(fft).max())
    fft_ar_sq = fft ** 2
    print(abs(fft_ar_sq).min(), abs(fft_ar_sq).max())
    N = len(ar) # this needs to be number of *Non zero* components
    out = fft_ar_sq * (dt / N)
    print(abs(out).min(), abs(out).max())
    # double non zero components to account for neg. frequencies
    out[1:] *= 2
    print(abs(out).min(), abs(out).max())
    freq = np.fft.rfftfreq(len(ar), dt)
    return freq, out

In [ ]:
freqs, fft = get_fft(source_disp, 1)
td_vel_num_diff = np.gradient(source_disp, dt)

td_vel_freq_diff = np.fft.irfft(fft * np.pi*2*1j*freqs)

plt.plot(td_vel_freq_diff, color='red')
plt.xlim(450, 550)


In [ ]:
plt.plot(td_vel_num_diff, color='blue')
plt.xlim(450, 550)

In [ ]:
print(np.sum(td_vel_num_diff**2), np.sum(td_vel_freq_diff**2))

In [ ]:
source_

In [ ]:
freqs, ppsd = get_ppsd(source_vel[:len(source_vel)//2], dt)
freqs, ppsd = get_ppsd(source_vel, dt)
plt.loglog(freqs, abs(ppsd))
plt.xlabel('frequency (hz)')
plt.ylabel('amplitude')

In [ ]:
spa = np.trapz(source_vel**2, dx=dt)
spb = np.sum(abs(get_ppsd(source_vel, dt)[1]))
assert np.isclose(spa, spb)

In [ ]:
print(spa, spb)